In [1]:
import imb_modeling
import pandas as pd
import numpy as np
import time

from sklearn import svm, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, roc_auc_score

from xgboost import XGBClassifier

Load dataset

In [2]:
svd8 = pd.read_csv('data/svd_8.csv',index_col=0)
svd_z5 = pd.read_csv("data/svd_z_5.csv",index_col=0)
svd_z9 = pd.read_csv("data/svd_z_9.csv",index_col=0)

In [3]:
svd8.shape, svd_z5.shape, svd_z9.shape

((6819, 9), (6819, 6), (6819, 10))

In [4]:
svd8.head()

,axis1,axis2,axis3,axis4,axis5,axis6,axis7,axis8,Bankrupt
0,5.995932e+09,2.565263e+09,1.998551e+09,-1.556894e+09,-2.872628e+09,-1.131120e+09,-6.199528e+08,-3.087006e+09,1
1,1.046412e+10,-2.025294e+09,7.646973e+09,2.906900e+09,-1.599878e+09,-1.508080e+09,-2.054188e+08,-2.352258e+09,1
2,5.855594e+09,-2.207877e+09,-2.404231e+09,-1.810148e+09,-2.697782e+09,-1.499563e+09,1.818762e+09,-3.908349e+08,1
3,1.071762e+10,1.341745e+09,6.819712e+09,7.430153e+08,-4.009937e+09,2.112051e+09,7.794637e+09,4.507580e+09,1
4,9.244816e+09,7.462440e+09,-1.875104e+09,3.030693e+09,-2.042418e+09,-3.254255e+08,-3.596034e+08,2.267177e+09,1


Params

In [5]:
# SVM parameter dictionary
SVM_params = {'kernel':('linear', 'rbf'), 
              'C':[1, 10]}

# Parameters for Decision treee
dt_params = {'criterion':['gini','entropy'], 
              'max_depth' : range(1,10), 
              'min_samples_split' : range(1,10), 
              'min_samples_leaf': range(1,5)}

# Parameters for Random Forest
rf_params = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)], # Number of trees in random forest
               'max_features': ['auto', 'sqrt'], # Number of features to consider at every split
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)], # Maximum number of levels in tree
               'min_samples_split': [2, 5, 10], # Minimum number of samples required to split a node
               'min_samples_leaf': [1, 2, 4], # Minimum number of samples required at each leaf node
               'bootstrap': [True, False] # Method of selecting samples for training each tree
                }

#Parameters for gradient boosting
gb_params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

#Parameters for XGBoost
xgb_estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    tree_method='gpu_hist',
    seed=421
)
xgb_params = {
    'max_depth': range(2, 10, 1),
    'n_estimators': range(60, 220, 40),
    "min_child_weight":[1, 3, 5],
    'learning_rate': [0.1, 0.01, 0.05]
}

In [6]:
models = [(svm.SVC(),SVM_params),
          (DecisionTreeClassifier(),dt_params),
          (RandomForestClassifier(),rf_params),
          (GradientBoostingClassifier(),gb_params),
          (xgb_estimator,xgb_params)]

In [7]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall':make_scorer(recall_score),
           'roc_auc':make_scorer(roc_auc_score)}

Modeling

In [17]:
res = []
for model, param in models[4:3:-1]:
    start = time.time()
    print('Training: ' + str(model))
    key_modified_params = {f"model__{key}": val for key, val in param.items()}
    output = imb_modeling.imb_pipe_fit(model,key_modified_params,X=svd8.drop(columns='Bankrupt'),y=svd8['Bankrupt'])
    res.append(output)
    end = time.time()
    print(output)
    print('Time: ' + str(end-start))

Training: XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, nthread=4, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, seed=421, subsample=None,
              tree_method='gpu_hist', validate_parameters=None, verbosity=None)
fitting model...
Fitting 30 folds for each of 288 candidates, totalling 8640 fits
